# Importing necessary Libraries

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import numpy as np                                
import pandas as pd
import urllib
import os

from sagemaker import get_execution_role


import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

from sagemaker import image_uris

In [2]:
from sagemaker.session import s3_input

# Data Sourse

In [3]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the ap-northeast-1 region. You will use the 501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [10]:
bucket_name = 'financemodel' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'northeast-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


# Create Features and Labels

In [5]:
try:
  model_data = pd.read_csv('./mycsvfile.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [6]:
#Set loan_status label as the first column in the dataset for sagermaker 
cols = list(model_data)
cols.insert(0, cols.pop(cols.index('loan_status')))
model_data = model_data.loc[:, cols]

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(model_data, test_size=0.2, random_state=25)

In [8]:
train_data.to_csv('train.csv', index=False, header=False)

In [11]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [12]:
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [22]:
sagemaker.image_uris.retrieve("xgboost", "ap-northeast-1",version='1.5-1')

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [23]:
container = image_uris.retrieve('xgboost', boto3.Session().region_name, '1')

In [24]:
sess = sagemaker.Session()

In [25]:
xgb = sagemaker.estimator.Estimator(image_uri=container,role=sagemaker.get_execution_role(), instance_count=1, instance_type='ml.m5.large',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)

In [26]:
xgb.set_hyperparameters(bjective='binary:logistic', eval_metric='error', verbosity=0, n_estimators = 500, max_depth = 6, learning_rate = 0.3, colsample_bytree = 0.5,num_round=100)

# Train Model

In [ ]:
xgb.fit({'train': s3_input_train})

In [27]:
trainedmodel = sagemaker.model.Model(
    model_data='s3://financemodel/sagemaker/DEMO-xgboost-dm/output/xgboost-2022-05-21-00-33-08-547/output/model.tar.gz',
    image_uri=container,  # example path for the semantic segmentation in
    role=role)  # your role here; could be different name



In [28]:
predictor = trainedmodel.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-----------!